In [1]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

from analysis import Subject, io, logger

logger.setLevel("INFO")
timer_subj_roots, control_subj_roots = io.load_path(Path("results/subjects"))

In [2]:
import numpy as np
from analysis.metrics import calculate_metrics
from analysis.consts import IDTConsts

idt_consts = IDTConsts(t_dur=0.1, t_disp=50)


path = timer_subj_roots[0]

subj = Subject(path, idt_consts=idt_consts)


def get_csv_line(subj: Subject):
    name = subj.name

    mfd_list = []
    mfd_sd_list = []
    for task_id in subj.task_order[1:]:
        trial_num = subj.task_id_to_trial_num(task_id)
        df_idt = subj.detect_fixations_idt(trial_num)
        mfd = df_idt["duration"].mean()
        mfd_sd = df_idt["duration"].std()
        mfd_list.append(float(mfd))
        mfd_sd_list.append(float(mfd_sd))

    mfd_overall = float(np.mean(mfd_list))
    mfd_sd_overall = float(np.mean(mfd_sd_list))

    csv_line = [name, *mfd_list, *mfd_sd_list, mfd_overall, mfd_sd_overall]
    return csv_line

In [3]:
header = ["Name"]
for task_id in range(1, 5):
    header.append(f"Task{task_id}_MFD")
for task_id in range(1, 5):
    header.append(f"Task{task_id}_MFD_SD")
header.append("Overall_MFD")
header.append("Overall_MFD_SD")

In [4]:
import csv

results = Path("./results") / "csv"
results.mkdir(exist_ok=True)
path = results / "with_timer.csv"
with path.open("w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=",")
    csv_writer.writerow(header)

for root in timer_subj_roots:
    subj = Subject(root, idt_consts=idt_consts)
    csv_line = get_csv_line(subj)
    with path.open("a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=",")
        csv_writer.writerow(csv_line)

In [5]:
results = Path("./results") / "csv"
results.mkdir(exist_ok=True)
path = results / "without_timer.csv"
with path.open("w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=",")
    csv_writer.writerow(header)

for root in control_subj_roots:
    subj = Subject(root, idt_consts=idt_consts)
    csv_line = get_csv_line(subj)
    with path.open("a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=",")
        csv_writer.writerow(csv_line)